In [1]:
import pandas as pd

import pickle
from tqdm import tqdm

In [2]:
test_pred = pickle.load(open("model_sep14_bioelectra_5lr_5epochs/bioelectrapreds.pkl", "rb"))

In [3]:
label_map = {'ACTIVATOR': 0, 'AGONIST': 1, 'AGONIST-ACTIVATOR': 2, 'AGONIST-INHIBITOR': 3, 'ANTAGONIST': 4,
 'DIRECT-REGULATOR': 5, 'INDIRECT-DOWNREGULATOR': 6, 'INDIRECT-UPREGULATOR': 7, 'INHIBITOR': 8,
 'PART-OF': 9, 'PRODUCT-OF': 10, 'SUBSTRATE': 11, 'SUBSTRATE_PRODUCT-OF': 12, 'false': 13}

In [4]:
label_map_index = { label_map[i]:i for i in label_map}
label_map_index

{0: 'ACTIVATOR',
 1: 'AGONIST',
 2: 'AGONIST-ACTIVATOR',
 3: 'AGONIST-INHIBITOR',
 4: 'ANTAGONIST',
 5: 'DIRECT-REGULATOR',
 6: 'INDIRECT-DOWNREGULATOR',
 7: 'INDIRECT-UPREGULATOR',
 8: 'INHIBITOR',
 9: 'PART-OF',
 10: 'PRODUCT-OF',
 11: 'SUBSTRATE',
 12: 'SUBSTRATE_PRODUCT-OF',
 13: 'false'}

In [5]:
test_pred_map = []
for pred in test_pred:
    test_pred_map.append(label_map_index[pred])


In [6]:
cols = ['pmid_vals', 'sentence']
df_test = pd.read_csv("processed_data/test.tsv", sep='\t', names=cols)

In [7]:
df_test[['PMID','arg1', 'arg2']] = df_test.pmid_vals.str.split(".",expand=True)

In [8]:
df_test['label'] = test_pred_map 

In [9]:
df_tosubmit = df_test[df_test['label'] != 'false']

In [10]:
df_tosubmit.head() 

,pmid_vals,sentence,PMID,arg1,arg2,label
6,32733640.T17.T30,"In moderately injured P19 neuronal cells, conc...",32733640,T17,T30,INHIBITOR
8,32733640.T18.T32,Western blot analysis revealed that @CHEMICAL$...,32733640,T18,T32,INDIRECT-DOWNREGULATOR
9,32733640.T18.T33,Western blot analysis revealed that @CHEMICAL$...,32733640,T18,T33,INDIRECT-DOWNREGULATOR
10,32733640.T18.T34,Western blot analysis revealed that @CHEMICAL$...,32733640,T18,T34,INDIRECT-UPREGULATOR
11,32733640.T18.T35,Western blot analysis revealed that @CHEMICAL$...,32733640,T18,T35,INDIRECT-UPREGULATOR


In [11]:
df_tosubmit.shape

(65489, 6)

In [12]:
pmid = df_tosubmit.PMID.tolist()
arg1 = df_tosubmit.arg1.tolist()
arg2 = df_tosubmit.arg2.tolist()

In [13]:
len(pmid)

65489

In [14]:
file = "drugprot-gs-training-development/test-background/test_background_entities.tsv"
text = open(file).read().splitlines()

data = [x.split("\t") for x in text]
df_test_entities = pd.DataFrame(data, columns= ["PMID", "arg", "entity", "start", "end", "name"])


In [15]:
df_test_entities.isnull().sum()


PMID      0
arg       0
entity    0
start     0
end       0
name      0
dtype: int64

In [16]:
new_arg1 = []
new_arg2 = []
for i in tqdm(range(len(pmid))):
#for i in range(100):
    check_entity = df_test_entities[(df_test_entities['PMID'] == pmid[i]) & (df_test_entities['arg'] == arg1[i])]['entity']
    #print(type(check_entity))
    if check_entity.values == 'GENE':
        new_arg1.append(arg2[i])
        new_arg2.append(arg1[i])
    else:
        new_arg1.append(arg1[i])
        new_arg2.append(arg2[i])


100%|██████████| 65489/65489 [39:31<00:00, 27.61it/s]


In [24]:
df_tosubmit.shape

(65489, 8)

In [22]:
len(new_arg1)

65489

In [19]:
df_tosubmit["new_arg1"] = new_arg1
df_tosubmit["new_arg2"] = new_arg2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [26]:
df_tosubmit.head(25)

,pmid_vals,sentence,PMID,arg1,arg2,label,new_arg1,new_arg2
6,32733640.T17.T30,"In moderately injured P19 neuronal cells, conc...",32733640,T17,T30,INHIBITOR,T17,T30
8,32733640.T18.T32,Western blot analysis revealed that @CHEMICAL$...,32733640,T18,T32,INDIRECT-DOWNREGULATOR,T18,T32
9,32733640.T18.T33,Western blot analysis revealed that @CHEMICAL$...,32733640,T18,T33,INDIRECT-DOWNREGULATOR,T18,T33
10,32733640.T18.T34,Western blot analysis revealed that @CHEMICAL$...,32733640,T18,T34,INDIRECT-UPREGULATOR,T18,T34
11,32733640.T18.T35,Western blot analysis revealed that @CHEMICAL$...,32733640,T18,T35,INDIRECT-UPREGULATOR,T18,T35
12,32733640.T19.T31,Western blot analysis revealed that quercetin ...,32733640,T19,T31,INDIRECT-UPREGULATOR,T19,T31
13,32733640.T19.T32,Western blot analysis revealed that quercetin ...,32733640,T19,T32,INDIRECT-UPREGULATOR,T19,T32
14,32733640.T19.T33,Western blot analysis revealed that quercetin ...,32733640,T19,T33,INDIRECT-UPREGULATOR,T19,T33
15,32733640.T19.T34,Western blot analysis revealed that quercetin ...,32733640,T19,T34,INDIRECT-UPREGULATOR,T19,T34
16,32733640.T19.T35,Western blot analysis revealed that quercetin ...,32733640,T19,T35,INDIRECT-UPREGULATOR,T19,T35


In [27]:
df_tosubmit["new_arg1"] = "Arg1:"+ df_tosubmit["new_arg1"]
df_tosubmit["new_arg2"] = "Arg2:"+ df_tosubmit["new_arg2"]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [28]:
df_final = df_tosubmit[['PMID','label', 'new_arg1','new_arg2']]

In [29]:
df_final

,PMID,label,new_arg1,new_arg2
6,32733640,INHIBITOR,Arg1:T17,Arg2:T30
8,32733640,INDIRECT-DOWNREGULATOR,Arg1:T18,Arg2:T32
9,32733640,INDIRECT-DOWNREGULATOR,Arg1:T18,Arg2:T33
10,32733640,INDIRECT-UPREGULATOR,Arg1:T18,Arg2:T34
11,32733640,INDIRECT-UPREGULATOR,Arg1:T18,Arg2:T35
...,...,...,...,...
239765,10772944,DIRECT-REGULATOR,Arg1:T2,Arg2:T10
239766,10772944,DIRECT-REGULATOR,Arg1:T2,Arg2:T12
239768,10772944,DIRECT-REGULATOR,Arg1:T3,Arg2:T15
239769,10772944,DIRECT-REGULATOR,Arg1:T3,Arg2:T16


In [33]:
df_final.to_csv("test_submission_sep15/bioelectra.tsv", sep="\t", index=False, header=None)